# CHAOS Stellar Mass Density Profiles

* [NGC0628](http://legacysurvey.org/viewer-dev?ra=24.1741665&dec=15.7834583&zoom=12&layer=decals-dr7&lslga)
* [NGC5194](http://legacysurvey.org/viewer-dev?ra=202.469547&dec=47.195151&zoom=11&layer=mzls+bass-dr6&lslga)
* [NGC5457](http://legacysurvey.org/viewer-dev?ra=210.802368&dec=54.349023&zoom=10&layer=mzls+bass-dr6&lslga)
* [NGC3184](http://legacysurvey.org/viewer-dev?ra=154.570590&dec=41.4243426&zoom=12&layer=mzls+bass-dr6&lslga)

In [24]:
import os, pdb
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import fitsio
from astropy.table import Column

In [25]:
import LSLGA.io
import LSLGA.misc
import LSLGA.qa

In [26]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2

In [27]:
%matplotlib inline

#### Specify the (SDSS) bands, the pixel scale, and the top-level data directory.

In [28]:
pixscale = 0.396 # [arcsec/pix]
band = ('g', 'r', 'i')

In [29]:
datadir = os.path.join( LSLGA.io.LSLGA_dir(), 'science', 'chaos' )

#### Read the sample.
Add some additional data we'll need to the output table.

In [30]:
def read_chaos(band=('g', 'r', 'i'), psfsize=1.3):
    """PSF size in FWHM arcsec is assumed to be constant.
    
    """
    sample = LSLGA.io.read_parent()
    these = np.hstack( [np.where(np.isin(sample['GALAXY'].data, chaosgal.encode('utf-8')))[0]
                        for chaosgal in ('NGC0628', 'NGC5194', 'NGC5457', 'NGC3184')] )
    sample = sample[these]

    # Add distances, the data release, the PSF widths, and the cutout size.
    sample.add_column(Column(name='DISTANCE', length=len(sample), dtype='f4'))
    dist = {'NGC0628': 7.2, 'NGC5194': 7.9, 'NGC5457': 7.4, 'NGC3184': 11.6} # [Mpc]    
    for ss in sample:
        ss['DISTANCE'] = dist[ss['GALAXY']] # [Mpc]
        
    sample.add_column(Column(name='DR', dtype='S3', length=len(sample)))
    gal2dr = {'NGC0628': 'DR7', 'NGC5194': 'DR6', 'NGC5457': 'DR6', 'NGC3184': 'DR6'}
    for ss in sample:
        ss['DR'] = gal2dr[ss['GALAXY']]
        
    # Cutout size (SDSS pixels).
    sample.add_column(Column(name='CUTOUT_SIZE', length=len(sample), dtype='i2'))
    gal2size = {'NGC0628': 2000, 'NGC5194': 2000, 'NGC5457': 3000, 'NGC3184': 1500}
    for ss in sample:
        ss['CUTOUT_SIZE'] = gal2size[ss['GALAXY']] # [SDSS pixels]
        
    # Add PSF width (nominal) and Galactic extinction values.
    for filt in band:
        sample['PSFSIZE_{}'.format(filt.upper())] = np.repeat(psfsize, len(sample))
    for filt in band:
        sample['MWDUST_A{}'.format(filt.upper())] = np.repeat(0.0, len(sample)) # initialize
    
    gal2dust = {'NGC0628': (0.232, 0.16, 0.119), # gri
                'NGC5194': (0.116, 0.08, 0.06), 
                'NGC5457': (0.028, 0.02, 0.015),
                'NGC3184': (0.055, 0.038, 0.028)}
    for ss in sample:
        for ii, filt in enumerate(band):
            ss['MWDUST_A{}'.format(filt.upper())] = gal2dust[ss['GALAXY']][ii]
        
    # Fix the radius and position angle of NGC5194
    #sample[]
        
    return sample

In [31]:
sample = read_chaos(band=band)
sample

LSLGA_ID,GALAXY,PGC,RA,DEC,TYPE,BAR,RING,MULTIPLE,COMPACTNESS,T,PA,D25,BA,DIAM_REF,Z,SB_D25,MAG,MAG_REF,WISE_RA,WISE_DEC,CNTR,W1MPRO,W1SIGMPRO,W2MPRO,W2SIGMPRO,W3MPRO,W3SIGMPRO,W4MPRO,W4SIGMPRO,RCHI2,CC_FLAGS,EXT_FLG,PH_QUAL,XSCPROX,W1RSEMI,W1BA,W1PA,W1GMAG,W1GERR,W2GMAG,W2GERR,W3GMAG,W3GERR,W4GMAG,W4GERR,IN_ALLWISE,IN_DESI,NEAR_BRIGHTSTAR,DISTANCE,DR,CUTOUT_SIZE,PSFSIZE_G,PSFSIZE_R,PSFSIZE_I,MWDUST_AG,MWDUST_AR,MWDUST_AI
int64,bytes29,int64,float64,float64,bytes4,bytes3,bytes3,bytes3,bytes3,float32,float32,float32,float32,bytes3,float32,float32,float32,bytes1,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float32,bytes4,int32,bytes4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,float32,bytes3,int16,float64,float64,float64,float64,float64,float64
518063,NGC0628,5974,24.174166500000002,15.783458300000001,Sc,nan,nan,nan,nan,5.2,nan,9.88553,0.94406086,iso,0.002196186,23.308481,9.705,B,24.1741387,15.7836868,234115101351047933,10.655,0.023,10.695,0.023,9.342,0.074,8.271,0.374,32.27,0000,5,AAAC,0.87,126.96,0.95,70.0,7.093,0.01,6.986,0.011,3.632,0.01,1.76,0.015,True,True,False,7.2,DR7,2000,1.3,1.3,1.3,0.232,0.16,0.119
1073844,NGC5194,47404,202.469547,47.195151,SABb,B,nan,M,nan,4.0,163.0,13.708821,0.85113806,iso,0.0015370633,22.921482,8.608,B,202.4696996,47.1951717,2029146901351042304,8.84,0.024,8.664,0.02,6.19,0.023,3.386,0.036,46.18,hhdd,5,AAAA,0.43,180.0,0.91,45.0,5.43,0.014,5.404,0.006,1.509,0.006,-0.28,0.03,True,True,False,7.9,DR6,2000,1.3,1.3,1.3,0.116,0.08,0.06
1237406,NGC5457,50063,210.80236799999997,54.349022999999995,SABc,B,nan,M,nan,5.9,nan,23.988337,0.9616123,iso,0.0007902133,23.88948,8.361,B,210.8021726,54.3487903,2119154501351013505,10.348,0.023,10.213,0.021,6.624,0.018,4.453,0.029,33.86,0000,5,AAAA,0.56,180.0,0.98,28.0,6.751,0.019,6.704,0.019,3.397,0.024,1.778,0.104,True,True,False,7.4,DR6,3000,1.3,1.3,1.3,0.028,0.02,0.015
1253132,NGC3184,30087,154.57058999999998,41.4243426,SABc,B,nan,nan,nan,5.9,nan,7.396052,0.97050995,iso,0.0019753666,23.384481,10.411,B,154.5705508,41.424279,1542140801351054772,10.97,0.023,10.88,0.021,7.418,0.021,4.711,0.03,23.9,0000,5,AAAA,0.69,115.66,0.97,60.0,7.652,0.011,7.517,0.011,4.133,0.008,2.339,0.017,True,True,False,11.6,DR6,1500,1.3,1.3,1.3,0.055,0.038,0.028


#### Download the SDSS imaging (if necessary).

In [32]:
def download_data(sample, pixscale=0.396, clobber=False):
    """Note that the cutout server has a maximum cutout size of 3000 pixels.
    
    montage -bordercolor white -borderwidth 1 -tile 2x2 -geometry +0+0 -resize 512 \
      NGC0628-SDSS.jpg NGC3184-SDSS.jpg NGC5194-SDSS.jpg NGC5457-SDSS.jpg chaos-montage.png
    """
    import subprocess
    
    for ss in sample:
        jpgfile = os.path.join(datadir, '{}-SDSS.jpg'.format(ss['GALAXY']))
        fitsfile = jpgfile.replace('.jpg', '.fits')
        if os.path.exists(jpgfile) and clobber is False:
            print('Done: {}'.format(jpgfile))
        else:
            cmd = 'wget -c -O {outfile}'
            cmd += '"http://legacysurvey.org/viewer/jpeg-cutout?ra={ra}dec={dec}&pixscale={pixscale}&size={size}&layer=sdss"'
            cmd = cmd.format(outfile=jpgfile, ra=ss['RA'], dec=ss['DEC'],
                             pixscale=pixscale, size=ss['CUTOUT_SIZE'])
            print(cmd)
            err = subprocess.call(cmd.split())
            
        if os.path.isfile(fitsfile) or not clobber:
            print('Done: {}'.format(fitsfile))
        else:
            cmd = cmd.replace('jpeg-cutout', 'fits-cutout')
            print(cmd)
            err = subprocess.call(cmd.split())

In [33]:
download_data(sample, pixscale=pixscale)

Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-SDSS.jpg
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-SDSS.fits
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-SDSS.jpg
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-SDSS.fits
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-SDSS.jpg
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-SDSS.fits
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-SDSS.jpg
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-SDSS.fits


#### Read the data, do ellipse-fitting, and write out the results.

In [34]:
def ellipse(xcen, ycen, semia, semib, phi, x, y):
    xp = (x-xcen) * np.cos(phi) + (y-ycen) * np.sin(phi)
    yp = -(x-xcen) * np.sin(phi) + (y-ycen) * np.cos(phi)
    return (xp / semia)**2 + (yp/semib)**2 <= 1

In [35]:
def read_multiband(onegal, band=('g', 'r', 'i'), refband='r', 
                   pixscale=0.396, verbose=False):
    """Read the multiband data and pack into a dictionary.
    
    """
    from astropy.stats import sigma_clipped_stats
    from scipy.ndimage.morphology import binary_dilation

    galaxy = onegal['GALAXY']
    
    imgfile = os.path.join(datadir, '{}-SDSS.fits'.format(galaxy))
    if verbose:
        print('Reading {}'.format(imgfile))
    img = fitsio.read(imgfile)
    sumimg = np.sum(img, axis=0)
    
    _, H, W = img.shape
    
    if galaxy == 'NGC5194':    
        #xcen, ycen = 1050, 925
        xcen, ycen = 1012, 995
        semia = 444 * 1.5
        semib = semia * (1 - 0.3)
        phi = np.radians(35 + 90)
    else:
        xcen, ycen = W // 2, H // 2
        #semia = 2 * onegal['W1RSEMI'] / pixscale
        semia = 0.85 * onegal['D25'] * 60 / 2 / pixscale
        semib = semia * onegal['BA']
        if np.isfinite(onegal['PA']):
            phi = np.radians(onegal['PA'])
            #phi = np.radians(onegal['W1PA'])
        else:
            phi = 0.0

    # Mask the main galaxy
    ymask, xmask = np.ogrid[0:H, 0:W] # mask the galaxy
    galmask = ellipse(xcen, ycen, semia, semib, phi, xmask, ymask)
    
    if galaxy == 'NGC5194': # also mask NGC5195
        xcen2, ycen2, rad2 = 850, 1650, 300
        ymask2, xmask2 = np.ogrid[-ycen2:H-ycen2, -xcen2:W-xcen2]
        mask2 = (xmask2**2 + ymask2**2) <= rad2**2
        mn, med, sig = sigma_clipped_stats(ma.masked_array(
            sumimg, np.logical_or(galmask, mask2)), sigma=2.5)
        mask = np.logical_or( np.logical_and(np.abs((sumimg - med)) > 2.5 * sig, ~galmask),
                              mask2 )
    else:
        mn, med, sig = sigma_clipped_stats(ma.masked_array(
            sumimg, galmask), sigma=2.5)
        mask = np.logical_and(np.abs((sumimg - med)) > 2.5 * sig, ~galmask)
        
    # Now populate the output dictionary
    data = dict()
    data['band'] = band
    data['refband'] = refband
    data['pixscale'] = pixscale
    
    for filt, indx in zip( data['band'], (0, 1, 2) ):
        # Correct for dust and convert to surface brightness--
        #print(onegal['GALAXY'], filt, 10**(0.4 * onegal['MWDUST_A{}'.format(filt.upper())]))
        sbimg = img[indx, :, :] * 10**(0.4 * onegal['MWDUST_A{}'.format(filt.upper())]) / data['pixscale']**2 # [nanomaggies/arcsec2]
        data[filt] = sbimg
        data['{}_mask'.format(filt)] = mask * 1
        data['{}_masked'.format(filt)] = ma.masked_array(sbimg * ~mask, mask)
        ma.set_fill_value(data['{}_masked'.format(filt)], 0)
        
    return data    

In [36]:
dd = read_multiband(sample[1])
#fitsio.write('junk.fits', dd['r_masked'], clobber=True)
#fitsio.write('junk2.fits', dd['r'], clobber=True)
#from LSLGA.mge import find_galaxy
#find_galaxy(dd['r_masked'], plot=True, quiet=False)

In [37]:
def _build_sbprofiles_one(args):
    """Wrapper function for the multiprocessing."""
    return build_sbprofiles_one(*args)

In [38]:
def build_sbprofiles_one(onegal, band=('g', 'r', 'i'), refband='r', 
                         pixscale=0.396, clobber=False, verbose=False):
    from LSLGA.ellipse import ellipsefit_multiband

    galaxy = onegal['GALAXY']
    print('Working on {}'.format(galaxy))
        
    #maxsma = np.round(onegal['CUTOUT_SIZE'] / 2).astype(int)
    maxsma = np.round(0.85 * onegal['D25'] * 60 / 2 / pixscale).astype(int) # [pixels]

    data = read_multiband(onegal, band=band, refband=refband,
                          pixscale=pixscale, verbose=verbose)
                
    # Read the ellipse-fitting results if they exist, otherwise do the fitting.
    ellipsefitfile = os.path.join(datadir, '{}-ellipsefit-fixed.p'.format(galaxy))
    if not os.path.exists(ellipsefitfile) or clobber:
        ell = ellipsefit_multiband(galaxy, datadir, data, onegal, maxsma=maxsma, 
                                   noellipsefit=True, verbose=verbose)
        LSLGA.io.write_ellipsefit(galaxy, datadir, ell, noellipsefit=True, 
                                  verbose=verbose)
    else:
        ell = LSLGA.io.read_ellipsefit(galaxy, datadir, noellipsefit=True, 
                                       verbose=verbose)
    return ell

In [39]:
def build_sbprofiles(sample, band=('g', 'r', 'i'), refband='r', 
                     pixscale=0.396, clobber=False, verbose=False):
    """Do ellipse-fitting and write out the surface brightness profiles.

    """
    from LSLGA.ellipse import ellipsefit_multiband

    sbargs = list()
    for onegal in sample:
        sbargs.append( (onegal, band, refband, pixscale, clobber, verbose) )

    if nproc > 1:
        p = multiprocessing.Pool(nproc)
        result = p.map(_build_sbprofiles_one, sbargs)
        p.close()
    else:
        result = list()
        for args in sbargs:
            result.append(_build_sbprofiles_one(sbargs))

In [40]:
%time build_sbprofiles(sample, clobber=True, verbose=True)

Working on NGC0628
Working on NGC5194
Working on NGC3184
Working on NGC5457
Reading /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-SDSS.fits
Reading /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-SDSS.fits
Reading /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-SDSS.fits
Reading /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-SDSS.fits
Finding the galaxy in the reference r-band image.
 Pixels used: 213939
 Peak Img[j, k]: 749 751
 Mean (j, k): 768.28 738.93
 Theta (deg): 97.2
 Astro PA (deg): 172.8
 Eps: 0.100
 Major axis (pix): 324.1
Finding the galaxy in the reference r-band image.
Finding the galaxy in the reference r-band image.
Finding the galaxy in the reference r-band image.
 Pixels used: 311829
 Peak Img[j, k]: 1000 1000
 Mean (j, k): 1017.18 989.85
 Theta (deg): 52.4
 Astro PA (deg): 37.6
 Eps: 0.331
 Major axis (pix): 423.0
 Pixels used: 366203
 Peak Img[j, k]: 1001 1001
 Mean (j, k): 996.89 995.07
 Theta (deg): 103.1
 Ast

In [41]:
def qa(sample, band=('g', 'r', 'i'), verbose=False):
    """Generate QA.
    
    """
    for onegal in sample:
        galaxy = onegal['GALAXY']
        data = read_multiband(onegal, band=band, verbose=verbose)
        ellipsefit = LSLGA.io.read_ellipsefit(galaxy, datadir, noellipsefit=True)
        
        png = os.path.join(datadir, '{}-ellipse-multiband.png'.format(galaxy))
        LSLGA.qa.display_multiband(data, ellipsefit=ellipsefit, png=png)
        
        smascale = onegal['DISTANCE'] * 1e3 / 206265
        png = os.path.join(datadir, '{}-ellipse-sbprofile.png'.format(galaxy))
        LSLGA.qa.display_ellipse_sbprofile(ellipsefit, png=png, smascale=smascale)

In [42]:
qa(sample, band=band)

Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-ellipse-multiband.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-ellipse-sbprofile.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-ellipse-multiband.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-ellipse-sbprofile.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-ellipse-multiband.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-ellipse-sbprofile.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-ellipse-multiband.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-ellipse-sbprofile.png


In [43]:
galaxy = 'NGC3184'
ss = sample[3]
ellipsefit = LSLGA.io.read_ellipsefit(galaxy, datadir, noellipsefit=True)
sbprofile = LSLGA.qa.ellipse_sbprofile(ellipsefit)

In [44]:
solarmag = {'g': 5.15, 'r': 4.67, 'i': 4.56} # absolute solar magnitude
area = ellipsefit['i'].sarea * pixscale**2 # area of elliptical sector [arcsec2]
print(area)
10**(-0.4 * (sbprofile['mu_i'] + 2.5 * np.log10(area) - 5 * np.log10(ss['DISTANCE'] * 1e3) + 5 - solarmag['i']))

[  0.313632     0.313632     0.313632     0.313632     0.313632
   0.313632     0.313632     0.313632     0.313632     0.313632
   0.313632     0.313632     0.313632     0.313632     0.313632
   0.313632     0.313632     0.313632     0.313632     0.313632
   0.313632     0.313632     0.313632     0.313632     0.313632
   0.313632     0.313632     0.313632     0.313632     0.313632
   0.313632     0.313632     0.313632     0.313632     0.313632
   0.56076643   0.67852738   0.82101813   0.99343194   1.20205265
   1.4544837    1.75992528   2.12950959   2.5767066    3.11781499
   3.77255613   4.56479292   5.52339944   6.68331332   8.08680912
   9.78503903  11.83989723  14.34518505  15.52277935  15.47337247
  15.57635727  15.71297062  15.77837113  15.90697016  15.84246958
  17.17413485  20.67663628  25.0187299   30.27266318  36.62992245
  44.32220617  53.62986946  64.89214205  78.51949188  95.00858393
 114.96038806 139.10206955]


array([1.90516611e+01, 1.90675602e+01, 1.90849657e+01, 1.91040108e+01,
       1.91248382e+01, 1.91476003e+01, 1.91724598e+01, 1.91995887e+01,
       1.92240388e+01, 1.78829810e+01, 1.78836432e+01, 1.78759243e+01,
       1.78377858e+01, 1.77867242e+01, 1.77295743e+01, 1.77633653e+01,
       1.74851901e+01, 1.73876895e+01, 1.70018575e+01, 1.67537750e+01,
       1.61395114e+01, 1.54652261e+01, 1.47048524e+01, 1.37157587e+01,
       1.27859462e+01, 1.18640971e+01, 1.09856996e+01, 1.00834516e+01,
       9.47307257e+00, 8.70887264e+00, 7.98620023e+00, 7.30944784e+00,
       6.65604378e+00, 6.06141389e+00, 5.53685058e+00, 2.87699566e+00,
       2.20171690e+00, 1.70025251e+00, 1.29576799e+00, 9.84663925e-01,
       7.60886886e-01, 5.80146816e-01, 4.49548859e-01, 3.48919470e-01,
       2.68378862e-01, 2.06350948e-01, 1.59140145e-01, 1.23426641e-01,
       9.55214265e-02, 7.50290102e-02, 5.85995631e-02, 4.52768438e-02,
       3.47048534e-02, 2.89502485e-02, 2.51309724e-02, 2.31481304e-02,
      

In [45]:
4 * np.pi * (ss['DISTANCE'] * 3.086e22)**2 * 10**(0.4 * (22.5 - sbprofile['mu_i']))

array([1.07239535e+50, 1.07329029e+50, 1.07427003e+50, 1.07534205e+50,
       1.07651440e+50, 1.07779566e+50, 1.07919497e+50, 1.08072202e+50,
       1.08209829e+50, 1.00661174e+50, 1.00664901e+50, 1.00621452e+50,
       1.00406775e+50, 1.00119356e+50, 9.97976659e+49, 9.99878710e+49,
       9.84220561e+49, 9.78732367e+49, 9.57014341e+49, 9.43050074e+49,
       9.08473906e+49, 8.70519188e+49, 8.27718660e+49, 7.72043748e+49,
       7.19705708e+49, 6.67815919e+49, 6.18371966e+49, 5.67585500e+49,
       5.33227992e+49, 4.90212086e+49, 4.49533715e+49, 4.11440128e+49,
       3.74660790e+49, 3.41189781e+49, 3.11662736e+49, 2.89549569e+49,
       2.68120834e+49, 2.50534696e+49, 2.31029305e+49, 2.12428716e+49,
       1.98623516e+49, 1.83245724e+49, 1.71813865e+49, 1.61358513e+49,
       1.50175939e+49, 1.39715274e+49, 1.30377467e+49, 1.22353689e+49,
       1.14576177e+49, 1.08895044e+49, 1.02910253e+49, 9.62112341e+48,
       8.93507241e+48, 8.06535500e+48, 6.97904470e+48, 6.47118071e+48,
      

In [46]:

ai, bi = 0.006, 1.114 # r-i vs i coefficients
ml = 10**(ai + bi * sbprofile['ri']) # M/L
print(ml)

[1.71042878 1.71323378 1.71629115 1.71962008 1.72324042 1.72717238
 1.73143628 2.12818535 2.13358971 1.74576767 1.75033543 1.75599872
 1.7627752  1.76970792 1.77735881 1.78082139 1.803346   1.80837964
 1.82370382 1.82388285 1.83531189 1.8507095  1.86728458 1.90052951
 1.94651861 2.02056926 2.10682011 2.20262298 2.4423881  2.38957235
 2.50093877 2.5589141  2.63438513 2.75892283 2.8222487  2.86085824
 2.92675428 2.95819176 3.07065579 3.04136076 3.05311071 3.04430245
 2.94948954 2.88713143 2.92043603 2.8397286  2.80349958 2.83081566
 2.7369597  2.68593242 2.63960447 2.58739211 2.51369556 2.61465151
 2.61750094 2.36959752 2.47635171 2.49319403 2.54815171 2.33009477
 2.36983812 2.28536577 2.15721178 2.16935717 2.19007805 2.09234899
 2.08496869 2.13035322 2.04138457 2.14786412 2.09950718 2.26159146]
